In [47]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv('Retained_Bank.csv')

In [50]:
df.head()

,Date,Package Name,Acquisition Channel,Store Listing Visitors,Installers,Visitor-to-Installer conversion rate,Installers retained for 1 day,Installer-to-1 day retention rate,Installers retained for 7 days,Installer-to-7 days retention rate,Installers retained for 15 days,Installer-to-15 days retention rate,Installers retained for 30 days,Installer-to-30 days retention rate
0,1/8/2019,B4 Bank,Organic,1628,414,0.254,359,0.867,344,0.831,335,0.809,330,0.797
1,1/8/2019,B4 Bank,Third-party referrers,514,46,0.089,38,0.826,35,0.761,32,0.696,30,0.652
2,1/8/2019,B4 Bank,Other,113,23,0.204,17,0.739,17,0.739,14,0.609,13,0.565
3,2/8/2019,B4 Bank,Organic,1610,407,0.253,354,0.870,338,0.830,330,0.811,322,0.791
4,2/8/2019,B4 Bank,Tracked channels (UTM),55,10,0.182,10,1.000,10,1.000,9,0.900,8,0.800


In [51]:
#Criação de um novo Dataframe cujo o canal de aquisição é o "Organic" 
df_organic = df.loc[df['Acquisition Channel']=='Organic']

In [52]:
df_organic.head()

,Date,Package Name,Acquisition Channel,Store Listing Visitors,Installers,Visitor-to-Installer conversion rate,Installers retained for 1 day,Installer-to-1 day retention rate,Installers retained for 7 days,Installer-to-7 days retention rate,Installers retained for 15 days,Installer-to-15 days retention rate,Installers retained for 30 days,Installer-to-30 days retention rate
0,1/8/2019,B4 Bank,Organic,1628,414,0.254,359,0.867,344,0.831,335,0.809,330,0.797
3,2/8/2019,B4 Bank,Organic,1610,407,0.253,354,0.870,338,0.830,330,0.811,322,0.791
7,3/8/2019,B4 Bank,Organic,1725,401,0.232,360,0.898,348,0.868,348,0.868,340,0.848
11,4/8/2019,B4 Bank,Organic,1579,410,0.260,365,0.890,350,0.854,341,0.832,330,0.805
15,5/8/2019,B4 Bank,Organic,1488,374,0.251,334,0.893,323,0.864,317,0.848,303,0.810


### Criação de um novo Dataframe, a partir do 'df_organic', apenas com as colunas que serão utilizadas para o modelo de Machine Learning

Este modelo utilizará as colunas "Store Listing Visitors" e "Installers" para, baseado nestes dados, prever quantas instalações ficarão retidas por 15 dias ("Installers retained for 15 days")

In [53]:
#Criação de um novo Dataframe (df2), a partir do 'df_organic', apenas com as colunas que serão utilizadas para o 
#modelo de Machine Learning

df2 = df_organic[['Store Listing Visitors', 'Installers', 'Installers retained for 15 days']]

In [54]:
df2.reset_index(inplace=True)

In [55]:
df2.drop('index',axis=1, inplace=True)

/home/vinicius/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [56]:
df2.head()

,Store Listing Visitors,Installers,Installers retained for 15 days
0,1628,414,335
1,1610,407,330
2,1725,401,348
3,1579,410,341
4,1488,374,317


In [57]:
#Separando df2 em colunas para treinamento do modelo (df_train) e variável dependente (df_target)

df_train = df2[['Store Listing Visitors', 'Installers']]
df_target = df2['Installers retained for 15 days']

## Padronização dos dados

In [58]:
from sklearn.preprocessing import StandardScaler

In [59]:
#Padronizando os dados..
#Neste caso, os dados utilizados para o treinamento serão colocados todos em um escala variando de -1 até 1, com
#média igual a zero.

scaler = StandardScaler()
df_train = scaler.fit_transform(df_train)

## Treinamento do modelo de Machine Learning

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
#Separando df_train em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_train, df_target, test_size=0.33, random_state=42)

In [62]:
#Importando o modelo de regressão linear
from sklearn.linear_model import LinearRegression

In [63]:
#Treinando o modelo...
reg = LinearRegression().fit(X_train, y_train)

In [64]:
#Fazendo a previsão...
y_pred_target = reg.predict(X_test)

In [65]:
#Cálculo dos erros cometidos pelo modelo..
#As métricas MAE e RMSE indicam o quão boa é a previsão, baseando-se nos erros cometidos pelo mesmo, ou seja,
#informa o quanto os dados previstos pelo modelo se distanciaram dos dados reais.

#Quanto mais os valores de erro se aproximam de 'zero', melhor..

from sklearn import metrics
print('MAE  (Erro Absoluto Médio:', metrics.mean_absolute_error(y_test, y_pred_target))
print()
print('RMSE (Raiz do erro quadrático médio):', np.sqrt(metrics.mean_squared_error(y_test, y_pred_target)))

MAE  (Erro Absoluto Médio: 8.15947705186076

RMSE (Raiz do erro quadrático médio): 9.736060200461779


In [66]:
#Cálculo do coeficente R2...
#Este coeficiente indica quão próximo estão os dados de previsão em relação aos dados reais. Assim, quanto mais
#próximo de 1 melhor...

from sklearn.metrics import r2_score
R_2 = r2_score(y_test, y_pred_target)  

print("Coeficiente de Determinação (R2):", R_2)

Coeficiente de Determinação (R2): 0.9898491169638777


### A análise das métricas MAE e RMSE, juntamente com o coeficiente de determinação R2, permitem inferir que o modelo tem um bom desempenho em predizer quanto dos clientes que instalaram o aplicativo em determinado dia ainda estarão utilizando o mesmo após 15 dias.

In [67]:
#Criação de um novo Dataframe (df_15_30) para indicar quanto dos clientes que permanceram com o aplicativo 
#instalado por 15 dias ainda o mantém por mais 15 dias..


df_15_30 = df_organic[['Installers retained for 15 days', 'Installers retained for 30 days']]

#alocando uma nova coluna indicando a diferença entre retidos até 15 dias e 30 dias..
df_15_30['não retidas após 15 dias'] = df_organic['Installers retained for 15 days']-df_organic['Installers retained for 30 days']

#calculando o percentual de diminuição entre 15 e 30 dias
df_15_30['percentual de queda'] = (1-(df_organic['Installers retained for 30 days']/df_organic['Installers retained for 15 days']))*100

/home/vinicius/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/vinicius/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [68]:
df_15_30.head()

,Installers retained for 15 days,Installers retained for 30 days,não retidas após 15 dias,percentual de queda
0,335,330,5,1.492537
3,330,322,8,2.424242
7,348,340,8,2.298851
11,341,330,11,3.225806
15,317,303,14,4.416404


In [69]:
#percentual médio de clientes que permaneceram por 15 dias, e entre 15 e 30 dias deixaram de utilizar o app...

print('percentual médio de desintalações após 15 dias: ')
print(round((sum(df_15_30['percentual de queda']))/len(df_15_30),2),'%')

percentual médio de desintalações após 15 dias: 
2.79 %


### O cálculo do percentual médio de desinstalações após 15 dias, permite inferir que menos de 3% dos usuários que instalaram o aplicativo, e permaneceram com o mesmo durante 15 dias, desinstalarão após este período.

### Desta forma, com este modelo é possível se prever quanto dos usuários que instalaram o aplicativo em determinado dia permanecerão com o mesmo instalado por 15 dias. Além disso, desta quantidade, em torno de 97% permanecerão com a aplicativo instalado até o trigésimo dia.